In [31]:
from tqdm import tqdm

In [32]:
import torch

In [33]:
import tensorflow as tf

In [34]:
# Load model directly
from transformers import AutoModel,AutoTokenizer
model = AutoModel.from_pretrained("ai4bharat/indic-bert")
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

In [35]:
param ={
    'maxLen' :256,
}

In [36]:
# keras.backend.set_image_data_format('channels_last')
# from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np

def pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0):
    padded_sequences = []
    for seq in sequences:
        if padding == 'pre':
            padded_seq = np.pad(seq, (maxlen - len(seq), 0), 'constant', constant_values=value)
        elif padding == 'post':
            padded_seq = np.pad(seq, (0, maxlen - len(seq)), 'constant', constant_values=value)
        else:
            raise ValueError("Padding should be 'pre' or 'post'.")

        if truncating == 'pre':
            padded_seq = padded_seq[-maxlen:]
        elif truncating == 'post':
            padded_seq = padded_seq[:maxlen]
        else:
            raise ValueError("Truncating should be 'pre' or 'post'.")

        padded_sequences.append(padded_seq)

    return np.array(padded_sequences, dtype=dtype)


def create_attention_masks(input_ids):
    attention_masks = []
    for seq in tqdm(input_ids):
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return np.array(attention_masks)

def getFeaturesandLabel(single_string, label):
    # Wrap the single string in a list
    sentences = ["[CLS] " + single_string + " [SEP]"]

    # Tokenize and preprocess
    tokenizer_texts = list(map(lambda t: tokenizer.tokenize(t)[:512], tqdm(sentences)))
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(tokenizer_texts)]

    # Pad sequences and create attention masks
    input_ids = pad_sequences(sequences=input_ids, maxlen=param['maxLen'], dtype='long', padding='post', truncating='post')
    attention_masks_data = create_attention_masks(input_ids)

    # Convert to torch tensors
    X_data = torch.tensor(input_ids)
    attention_masks_data = torch.tensor(attention_masks_data)
    y_data = torch.tensor(label)

    return X_data, attention_masks_data, y_data


In [71]:
import pickle
with open('classifier.pkl','rb') as file:
  clf=pickle.load(file)

In [83]:
text_input="థమ్స్ అప్ ఓకే బాయ్"
# from app import predict
# text_input=predict.input_string
label_input = [1]
X_data, attention_masks_data, y_data = getFeaturesandLabel(text_input, label_input)

100%|██████████| 1/1 [00:00<00:00, 1562.12it/s]


In [84]:
torch.save(X_data, 'X_data')

In [85]:
def get_tensor_variable():
    # Define a PyTorch tensor variable
    return X_data

# Save the variable to a file (optional, but can be useful)
torch.save(get_tensor_variable(), 'X_data.pth')


In [86]:
print(X_data)

tensor([[     2,  29913,   3780,   4491,  24484,    884,   1399, 129604,      8,
              3,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,    

In [87]:
prediction = clf.predict(X_data)
print(prediction)

[1]
